In [3]:
import cv2
import numpy as np

def estimate_noise_variance(image):
    # Estimate the noise variance from the corners of the image
    h, w = image.shape
    corner_size = 50  # You can adjust this based on your image size
    corners = [image[:corner_size, :corner_size],  # Top-left corner
               image[:corner_size, -corner_size:],  # Top-right corner
               image[-corner_size:, :corner_size],  # Bottom-left corner
               image[-corner_size:, -corner_size:]]  # Bottom-right corner
    variances = [np.var(corner) for corner in corners]
    return np.mean(variances)

def deblur_wiener(blurred_image, psf, noise_var):
    # Compute the Fourier transforms of the blurred image and the PSF
    blurred_image_fft = np.fft.fft2(blurred_image)
    psf_fft = np.fft.fft2(psf, s=blurred_image.shape)

    # Precompute constant values
    psf_abs_squared = np.abs(psf_fft) ** 2
    regularization_lambda = 1e-3  # Adjust this regularization parameter
    wiener_filter = np.conj(psf_fft) / (psf_abs_squared + noise_var + regularization_lambda)

    # Apply the Wiener filter in the frequency domain
    deblurred_image_fft = blurred_image_fft * wiener_filter

    # Compute the inverse Fourier transform to obtain the deblurred image
    deblurred_image = np.fft.ifft2(deblurred_image_fft).real

    return deblurred_image


# Load the blurred image
blurred_image = cv2.imread('blurred_image.jpg', cv2.IMREAD_GRAYSCALE)

# Estimate the noise variance from the image
noise_var = estimate_noise_variance(blurred_image)

# Define the point spread function (PSF) - You need to provide this based on your specific case
# Example PSF: psf = np.array([[0.05, 0.2, 0.05], [0.2, 0.5, 0.2], [0.05, 0.2, 0.05]])
psf = np.array([[0.05, 0.2, 0.05], 
                [0.2, 0.5, 0.2],
                [0.05, 0.2, 0.05]])
# Deblur the image using the Wiener filter with regularization
deblurred_image = deblur_wiener(blurred_image, psf, noise_var)

# Clip to ensure pixel values are in the valid range [0, 255]
deblurred_image = np.clip(deblurred_image, 0, 255).astype(np.uint8)

# Save or display the deblurred image
cv2.imwrite('deblurred_image_wiener.jpg', deblurred_image)
cv2.imshow('Deblurred Image (Wiener)', deblurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()